In [1]:
import dask.dataframe as dd
import dask.array as da
import glob
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error, mean_squared_log_error, r2_score

In [2]:
parq = glob.glob("big_processed_4.parquet/*.parquet")
ddf = dd.read_parquet(parq)

In [3]:
dd_train, dd_test = ddf.random_split([0.80, 0.2], random_state = 2019)

In [4]:
df = dd_train.compute()

In [5]:
neighbor_cols = [f'neighbor_{n}' for n in range(25)]
df[neighbor_cols] = pd.DataFrame(df.neighbors.values.tolist(), index= df.index)

In [6]:
# even though we imputed by timestamp in the spark file, some timestamps were wholly empty
mean_epa = df['imputed_epa_pm25_value'].mean()
df['imputed_epa_pm25_value'] = df['imputed_epa_pm25_value'].fillna(mean_epa)

In [7]:
df.shape

(9752724, 34)

In [10]:
df.isnull().any()

2_5um                     False
imputed_epa_pm25_value    False
imputed_hum               False
imputed_temperature       False
wind_x                    False
wind_y                    False
ndvi                      False
elevation                 False
neighbors                 False
neighbor_0                False
neighbor_1                False
neighbor_2                False
neighbor_3                False
neighbor_4                False
neighbor_5                False
neighbor_6                False
neighbor_7                False
neighbor_8                False
neighbor_9                False
neighbor_10               False
neighbor_11               False
neighbor_12               False
neighbor_13               False
neighbor_14               False
neighbor_15               False
neighbor_16               False
neighbor_17               False
neighbor_18               False
neighbor_19               False
neighbor_20               False
neighbor_21               False
neighbor

In [18]:
input_cols = [col for col in df.columns if col != "neighbors"]

In [20]:
df[input_cols].to_parquet("final_train_2.parquet")

In [21]:
import gc
del df
gc.collect()

2407

In [22]:
df = dd_test.compute()
df[neighbor_cols] = pd.DataFrame(df.neighbors.values.tolist(), index= df.index)
df['imputed_epa_pm25_value'] = df['imputed_epa_pm25_value'].fillna(mean_epa)
df.shape

(2440650, 34)

In [23]:
df[input_cols].to_parquet("final_test_2.parquet")